In [25]:
import pandas as pd
import requests

import openmeteo_requests
import requests_cache
from retry_requests import retry

### Get main CSV

In [26]:
CSV_POLEN = r"..\new_datasets\datos_gramineas.csv"
df_master = pd.read_csv(CSV_POLEN)
df_master['fecha'] = pd.to_datetime(df_master['fecha'])

### Get new Polen

In [27]:
URL_POLEN = "https://datos.comunidad.madrid/catalogo/dataset/e608aace-3593-43a3-8c91-02332137fa83/resource/db5e3952-57f5-40f3-bb1d-906eb17aebb1/download/mediciones_polen.json"

response = requests.get(URL_POLEN)
nuevos_datos = response.json()
df_nuevos = pd.DataFrame(nuevos_datos['data'] if 'data' in nuevos_datos else nuevos_datos)

df_nuevos = df_nuevos[df_nuevos['tipo_polinico'] == 'Gramíneas']
df_nuevos = df_nuevos[df_nuevos['captador'] == 'AYTM']
df_nuevos['fecha'] = pd.to_datetime(df_nuevos['fecha_lectura'])
df_polen = df_nuevos[['fecha', 'granos_de_polen_x_metro_cubico']].rename(columns={'granos_de_polen_x_metro_cubico': 'granos_de_polen_x_metro_cubico'})


### Get new Meteo

In [28]:
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

URL_METEO = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 40.4165,
	"longitude": -3.7026,
	"hourly": ["temperature_2m", "wind_speed_10m", "wind_gusts_10m", "relative_humidity_2m", "wind_direction_10m", "dew_point_2m", "rain", "vapour_pressure_deficit", "et0_fao_evapotranspiration", "cloud_cover", "shortwave_radiation"],
	"past_days": 31,
}
responses = openmeteo.weather_api(URL_METEO, params=params)

response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(1).ValuesAsNumpy()
hourly_wind_gusts_10m = hourly.Variables(2).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(3).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(4).ValuesAsNumpy()
hourly_dew_point_2m = hourly.Variables(5).ValuesAsNumpy()
hourly_rain = hourly.Variables(6).ValuesAsNumpy()
hourly_vapour_pressure_deficit = hourly.Variables(7).ValuesAsNumpy()
hourly_et0_fao_evapotranspiration = hourly.Variables(8).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(9).ValuesAsNumpy()
hourly_shortwave_radiation = hourly.Variables(10).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end =  pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m (°C)"] = hourly_temperature_2m
hourly_data["wind_speed_10m (km/h)"] = hourly_wind_speed_10m
hourly_data["wind_gusts_10m (km/h)"] = hourly_wind_gusts_10m
hourly_data["relative_humidity_2m (%)"] = hourly_relative_humidity_2m
hourly_data["wind_direction_10m (°)"] = hourly_wind_direction_10m
hourly_data["dew_point_2m (°C)"] = hourly_dew_point_2m
hourly_data["rain (mm)"] = hourly_rain
hourly_data["vapour_pressure_deficit (kPa)"] = hourly_vapour_pressure_deficit
hourly_data["et0_fao_evapotranspiration (mm)"] = hourly_et0_fao_evapotranspiration
hourly_data["cloud_cover (%)"] = hourly_cloud_cover
hourly_data["shortwave_radiation (W/m²)"] = hourly_shortwave_radiation

hourly_dataframe = pd.DataFrame(data = hourly_data)

hourly_dataframe['fecha'] = pd.to_datetime(hourly_dataframe['date']).dt.date
df_meteo = hourly_dataframe.groupby('fecha').mean(numeric_only=True).reset_index()
df_meteo['fecha'] = pd.to_datetime(df_meteo['fecha'])

Coordinates: 40.4375°N -3.6875°E
Elevation: 651.0 m asl
Timezone difference to GMT+0: 0s


### Get new Contaminantes

In [29]:
URL_CONTAMINANTES = "https://datos.madrid.es/egob/catalogo/300755-12751586-calidad-aire-tiempo-real-acumula.json"

response = requests.get(URL_CONTAMINANTES)
nuevos_datos = response.json()
df_nuevos = pd.DataFrame(nuevos_datos['data'] if 'data' in nuevos_datos else nuevos_datos)
df_nuevos = pd.DataFrame(df_nuevos['records'].tolist())

df_nuevos_cont = df_nuevos[df_nuevos['ESTACION'] == "8"].copy()

magnitude_map = {
    '1': 'SO2 (ug/m3)', '6': 'CO (mg/m3)', '7': 'NO (ug/m3)', '8': 'NO2 (ug/m3)',
    '9': 'PM2.5 (ug/m3)', '10': 'PM10 (ug/m3)', '12': 'NOx (ug/m3)', '14': 'O3 (ug/m3)',
    '20': 'Tolueno (ug/m3)', '30': 'Benceno (ug/m3)', '42': 'HCT (mg/m3)', '44': 'HCNM (mg/m3)'
}

d_cols = [f'H{i:02d}' for i in range(1, 25)]
v_cols = [f'V{i:02d}' for i in range(1, 25)]

rows = []
for idx, row in df_nuevos_cont.iterrows():
    for d, v in zip(d_cols, v_cols):
        if v in row and row[v] == 'V':
            val = str(row[d]).replace(',', '.')
            hour_num = int(d[1:])
            rows.append({
                'ano': row['ANO'], 
                'mes': row['MES'], 
                'dia': row['DIA'],
                'hora': hour_num,
                'magnitud': row['MAGNITUD'],
                'valor': pd.to_numeric(val, errors='coerce')
            })

# Nueva columna fecha
df_melted = pd.DataFrame(rows)
df_melted['fecha'] = pd.to_datetime(df_melted[['ano', 'mes', 'dia']].rename(
    columns={'ano': 'year', 'mes': 'month', 'dia': 'day'}), errors='coerce')
df_melted = df_melted.dropna(subset=['fecha'])

# Hacer media
df_diario = df_melted.groupby(['ano', 'mes', 'dia', 'magnitud'], as_index=False)['valor'].mean()
df_diario['fecha'] = pd.to_datetime(df_diario[['ano', 'mes', 'dia']].rename(
    columns={'ano': 'year', 'mes': 'month', 'dia': 'day'}
)).dt.date

# Reformatear tabla
df_diario['magnitud'] = df_diario['magnitud'].astype(str).map(magnitude_map)
df_nuevos_cont = df_diario.pivot_table(
    index='fecha', 
    columns='magnitud', 
    values='valor', 
    aggfunc='mean'
).reset_index()
df_nuevos_cont.columns.name = None
df_nuevos_cont['fecha'] = pd.to_datetime(df_nuevos_cont['fecha'])

### MIX

In [30]:
df_updates = pd.merge(df_polen, df_meteo, on='fecha', how='outer')
df_updates = pd.merge(df_updates, df_nuevos_cont, on='fecha', how='outer')
df_final = pd.concat([df_master, df_updates], ignore_index=True)
df_final = df_final.groupby('fecha', as_index=False).first()

df_final = df_final.sort_values('fecha', ascending=False)

df_final.to_csv(r"..\new_datasets\datos_gramineas.csv", index=False)